# Sandoz - Accumulations

In [1]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

## Import PDF

In [2]:
# Read pdf into DataFrame
df = tabula.read_pdf("payment-disclosure-sandoz-switzerland-2018-de.pdf", pages='all', lattice=True)

## Format Table

In [3]:
df_export = df.copy()

#Drop Columns
df_export = df_export.drop(df_export.columns[9:],axis=1)
df_export = df_export.drop(df_export.columns[[7]],axis=1)

#Rename
df_export.columns = fix_columns_accumulations[0: 8]

#Remove char '\uf0d8'
df_export =  df_export.replace('\uf0d8', ' ', regex=True)

amount = df_export[df_export.type.str.contains('Diesen Empfängern zuzuordnender ', na=False)]
recipients = df_export[df_export.type.str.contains('Zahl der Empfänger', na=False)]
rnd = df_export[df_export.type.str.contains('Geldwerte Leistungen in', na=False)]

df_export = pd.concat([amount, recipients, rnd])

#Numberize
df_export = replace_comma_to_dot(df_export)
df_export = amounts_to_number(df_export)

#Rename first column
df_export = df_export.reset_index(drop=True)
df_export.iloc[0,0] = "hcp_amount"
df_export.iloc[1,0] = "hco_amount"
df_export.iloc[2,0] = "hcp_count"
df_export.iloc[3,0] = "hco_count"
df_export.iloc[4,0] = "rnd"
df_export.loc[df_export.type == 'rnd', "total"] = df_export['donations_grants']
df_export.loc[df_export.type == 'rnd', "donations_grants"] = np.NaN
df_export.loc[df_export.type == 'rnd', "sponsorship"] = np.NaN

#Export
export_acumulations(df_export, 'sandoz')

Total nicht Summe der Werte
saved


In [4]:
df_export

,type,donations_grants,sponsorship,registration_fees,travel_accommodation,fees,related_expenses,total,source
0,hcp_amount,NaN,NaN,NaN,NaN,9095.17,919.36,10014.53,sandoz
1,hco_amount,78729.5,14545.2,NaN,NaN,600.00,NaN,93874.70,sandoz
2,hcp_count,NaN,NaN,NaN,NaN,9.00,2.00,NaN,sandoz
3,hco_count,1.0,5.0,NaN,NaN,1.00,NaN,NaN,sandoz
4,rnd,NaN,NaN,NaN,NaN,NaN,NaN,693.59,sandoz


In [ ]:
#write_to_csv(df_export, 'tmp.csv', True)
#write_to_excel(df_export, 'tmp.xlsx')